Getting latest news on crypto and stocks for my personal tracking purposes.

for crypto: I will use https://decrypt.co/
for stocks: I will use yahoo finance api

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

Now we have to have a closer look at the decrypt website for proper scraping and analysing of the data.

In [4]:
# A class to represent a Webpage - decrypt
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Block for Yahoo Finance API


In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:
system_prompt = "You are an assistant that analyzes the contents of a cryptocurrency  website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."
def user_prompt_for_crypto(website):
    user_prompt = f"You are looking at a cryptocurrencies website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements about any coins' prices going up or down, then summarize these too. \
Also please make a table of the prices of the different coins prices and their percentage change. Format the table nicely so that every coin shows on a separate row of its own.\n\n"
    user_prompt += website.text
    return user_prompt
    

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for_crypto(website)}
    ]

In [ ]:
def summarize_crypto(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:

def display_summary(url):
    summary = summarize_crypto(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://decrypt.co/")

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:
system_prompt_s = "You are an assistant that analyzes the contents of yahoo finance website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."
def user_prompt_for_stocks(website):
    user_prompt = f"You are looking at a yahoo finance website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements about any stocks around the world going up or down, any world crisis that might affect the stock market. \
Also please make a table of the prices of the top 10 stock and etfs going up or down around the world. \
Also ignore all the news related to the crypto, and focus only on the stocks \
Please analyse all the news and check if there's a predicted shift in specific stock prices.\n\n"
    user_prompt += website.text
    return user_prompt

def messages_for_stocks(website):
    return [
        {"role": "system", "content": system_prompt_s},
        {"role": "user", "content": user_prompt_for_stocks(website)}
    ]


def summarize_stocks(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for_stocks(website)
    )
    return response.choices[0].message.content
    
def display_summary_stocks(url):
    summary = summarize_stocks(url)
    display(Markdown(summary))
    
display_summary_stocks("https://finance.yahoo.com/")

# Yahoo Finance Summary

Yahoo Finance serves as a comprehensive platform for tracking the stock market, offering live quotes and business news. The site provides in-depth analysis of recent financial trends, including notable movements in major stock indices and individual stocks.

## Key News Highlights
- **Tesla (TSLA)** aims for a record close after a significant uptrend, surging by 5.28%.
- **Nvidia (NVDA)** also saw gains of 3.45%, reflecting investor confidence amidst tensions related to the US-China tech war.
- The market reacted positively to inflation data, indicating potential for a Federal Reserve interest rate cut next week.

### Global Market Impacts
- The market is witnessing fluctuations due to various external factors, including economic indicators and geopolitical tensions.
- Analysts predict a bullish market outlook and highlight sectors to watch in 2025 for ETF investors. 

## Stock Price Movements
Here’s a summary of the top 10 stocks and ETFs that are experiencing significant changes today:

| Stock/ETF                  | Price    | Change (%)    |
|---------------------------|----------|---------------|
| **Top Gainers**           |          |               |
| Patterson Companies (PDCO)| $31.06   | +34.37%       |
| ReNew Energy (RNW)       | $7.48    | +17.98%       |
| Rigetti Computing (RGTI)  | $7.29    | +12.25%       |
| JetBlue Airways (JBLU)   | $7.54    | +11.14%       |
| **Top Losers**            |          |               |
| Bausch + Lomb (BLCO)     | $18.06   | -12.54%       |
| IonQ, Inc. (IONQ)        | $30.07   | -8.81%        |
| Bausch Health (BHC)      | $7.61    | -8.59%        |
| CG Oncology (CGON)       | $31.18   | -7.47%        |
| C3.ai, Inc. (AI)        | $38.70   | -7.27%        |

### Predicted Shifts in Stock Prices
- Positive expectations are set for **airline stocks** and large tech companies given the recent CPI report.
- The outlook for **American companies** like Apple (AAPL) remains mixed with a slight decline in price, indicating a potential volatility in future valuations.

Overall, the Yahoo Finance platform continues to provide essential insights and timely updates for investors navigating the stock market.